# Genarate Stocks Demo Data

Run the code below to generate the key/value table, time-series table and stream used in the demo<br>
> ### In order to run the notebook you need to obtain a free token from WorldTradingData 

### Require obtaining a (free) API token from [World Trading Data](https://www.worldtradingdata.com) and setting the environment variable below 

In [1]:
%env API_TOKEN = <Insert world trading data token>

env: API_TOKEN=<Insert world trading data token>


In [2]:
# import required libraries 
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sys
sys.path.append('../../')
import v3io_frames_local as v3f

In [3]:
# initialize iguazio multi-model DB dataframe client library 
client = v3f.Client('framesd:8081')

## Delete KV, TSDB, and Stream tabels
in case we want to start things from scratch (delete current tabels), uncomment the following line(s) and run them 

In [4]:
#client.delete('kv','stocks')

In [5]:
#client.delete('tsdb','stock_metrics')

In [6]:
#client.delete('stream','stock_stream')

## Create TSDB, KV, and Stream tabels 

In [7]:
client.create(backend='tsdb', table='stock_metrics',attrs={'rate':'1/m'})

ValueError: Protocol message CreateRequest has no "attrs" field.

In [ ]:
client.create(backend='stream', table='stock_stream',attrs={'retention_hours':48,'shards':1})

In [ ]:
# fill the key/value table with some data (KV tables are automatically created on write and have a dynamic schema)
kvtbl = '{"price":{"GOOG":1039.55,"AMZN":1641.03,"AAPL":169.6,"MSFT":107.59,"INTC":47.21},"volume":{"GOOG":1807725,"AMZN":7494808,"AAPL":62025994,"MSFT":40801525,"INTC":23289000},"symbol":{"GOOG":"GOOG","AMZN":"AMZN","AAPL":"AAPL","MSFT":"MSFT","INTC":"INTC"},"exchange":{"GOOG":"NASDAQ","AMZN":"NASDAQ","AAPL":"NASDAQ","MSFT":"NASDAQ","INTC":"NASDAQ"},"last_trade":{"GOOG":"2018-12-10 16:00:01","AMZN":"2018-12-10 16:00:02","AAPL":"2018-12-10 16:00:02","MSFT":"2018-12-10 16:00:02","INTC":"2018-12-10 16:00:02"},"name":{"GOOG":"Alphabet Inc Class C","AMZN":"Amazon.com, Inc.","AAPL":"Apple Inc.","MSFT":"Microsoft Corporation","INTC":"Intel Corporation"},"currency":{"GOOG":"USD","AMZN":"USD","AAPL":"USD","MSFT":"USD","INTC":"USD"},"timezone":{"GOOG":"EST","AMZN":"EST","AAPL":"EST","MSFT":"EST","INTC":"EST"}}'
client.write(backend='kv', table='stocks',dfs=pd.read_json(kvtbl))

## Fill the time-series table with a week worth or historical data from WorldTradingData API 

In [ ]:
# read the stocks kv table (to get the Symboles)
sdf = client.read(backend='kv', table='stocks') 
stocklist = sdf.index.tolist()

# create all stocks data based on stocks table & WTD history API
# need the symbol & exchange name from stocks table 
urlt = 'https://www.worldtradingdata.com/api/v1/intraday?symbol={0}&range=7&sort=asc&interval=1&output=csv&api_token=' + os.getenv('API_TOKEN')
for sym in stocklist:
    if not sym:
        continue 
    url = urlt.format(sym)
    df = pd.read_csv(url,skiprows=[0])
    df.drop(['Open','High','Low'], axis=1, inplace=True)
    df.rename(columns={'Close': 'price', 'Volume': 'volume'}, inplace=True)
    
    # generate random sentiment series per stock
    df['sentiment'] = np.random.uniform(low=0.0, high=2, size=(len(df),))-1
    
    # set the index to date, symbol, exchange (will be marked as TSDB labels) 
    df.Date = pd.to_datetime(df.Date)
    df['exchange']=sdf.loc[sym].exchange
    df['symbol']=sym
    newdf =df.set_index(['Date','symbol','exchange'])
    
    # write to the TSDB
    print(newdf.head())
    client.write(backend='tsdb', table='stock_metrics',dfs=newdf)

## Fill dummy tweet data in the stream

In [ ]:
import json
record = {'text': 'bla bla bla', 
          'user': '@supermen',
          'id': 1102722594429132545,
          'created_at':'Tue Mar 02 00:08:48 +0000 2019',
          'polarity':0.3,
          'subjectivity':0.1,
         }
client.execute('stream', 'stock_stream', 'put', args={'data': json.dumps(record)})